### Scribble editor and result display 
#### display of scribbles in slicer, possible manual scribble update in slicer, conversion of scribbles to seeds for graphcut (initially 2D only),invoking graphcut, displaying result of graphcut in slicer, iteration in slicer, save result

#### Import

In [60]:
import os
import glob
import sys

import shutil
import json

import numpy as np
import random
import matplotlib.pyplot as plt
import scipy
from scipy import ndimage
import nibabel as nib
from scipy.ndimage import morphology
import SimpleITK
import pandas as pd
import JupyterNotebooksLib as slicernb

### Interactive scribble and graphcut Plan
#### Choose (a) Patient name, srcDir, expDir, patConfig, graphCut config (b) Create random scribbles; (c) Load in slicer : Volumes (CT, PET, softmax), Segmentations (ground Truth:  gt, prediction from NN: predNN, foreground scribble: fgScribble, background-scribble: bgScribble, initial graph cut results using CT, PET and Softmax) (d) User modifies fgScribble and bgSScribble using slicer (e) Use Jupyter NB to save updated fgScribble and bgScribble (f) Repeat graphcut using jupyter - graph cut results using CT, PET and Softmax; (f) View in slice -> go back to step(d)

### Helper functions 
#### Most of these has already been tested in other notebook(s); 

In [61]:
sys.path.append('/home/user/DMML/CodeAndRepositories/MMGTVSeg')
import src
from src import scribbleHelper, gcHelper, graphCutClient, graphCutServer, libclient, libserver
from src.scribbleHelper import checkFolderExistenceAndCreate, readAndScaleImageData,\
    dice_coef_func, dice_multi_label, disk, ball, bbox2_3D, getUnionBoundingBoxWithPadding,\
    chooseScribbleFromMissedFGOrWrongCBG2D, chooseScribbleFromDefiniteRegion2D,\
    autoGenerateScribbleRegions2D,\
    chooseScribbleFromMissedFGOrWrongCBG3D, chooseScribbleFromDefiniteRegion3D,\
    autoGenerateScribbleAndBBox3D,\
    createGCInputUsingGT
from src.gcHelper import remote_generateGrahcutSegmentationAndDiceFromJson 
# import src.gcHelper as gcHelper

#### Initial input and config

In [62]:
#Data and experiment folder and initial patient name
srcFolder =\
'/home/user/DMML/Data/HeadNeck_PET_CT/nnUnet_3dfullres/validation_gtvs_withSoftmax'
expFolder = '/home/user/DMML/Data/PlayDataManualSegmentation/AutoScribbleExperiment'
# srcFolder = 'J:/HecktorData/nnUnet_3dfullres/validation_gtvs_withSoftmax'
# expFolder = 'J:/PlayDataManualSegmentation/AutoScribbleExperiment'

patientName = 'CHUM038'
expPatName = "expPat"

#Configs
patDataConfig = \
{   
     "ctSuffix": "_ct.nii.gz",
     "ptSuffix": "_pt.nii.gz",
     "gtSuffix": "_ct_gtvt.nii.gz",
     "predSuffix": "_segment.nii.gz",
     "softmaxSuffix": "_softMax.nii.gz",
     "fgScribbleSuffix": "_fgScribble.nii.gz",
     "bgScribbleSuffix": "_bgScribble.nii.gz",
     "ct_gCutSuffix": "_ct_gCut.nii.gz",
     "pet_gCutSuffix": "_pet_gCut.nii.gz",         
     "softmax_gCutSuffix": "_softmax_gCut.nii.gz",
     "ct_low": -1000,
     "ct_high": 3095,
     "ct_clipImages": False,
     "ct_scaleImages": False,
     "ct_scaleFactor": 1000,
     "ct_normalizeImages": False,
     "pt_low": -1000,
     "pt_high": 3095,
     "pt_clipImages": False,
     "pt_scaleImages": False,
     "pt_scaleFactor": 1,
     "pt_normalizeImages": False,
     "labels_to_train": [1]
}


autoScribbleAndGCConfig = \
{
 'bbPad' : 2,
 'fractionDivider' : 10,
 'dilation_diam' : 2,
 'useAtmostNScribblesPerRegion' : 4,
 'segparams_ssgc' : 
        {
         "method": "graphcut",
         "pairwise_alpha": 1.0,
         "modelparams": 
            {
             "cvtype": "full",
             "params": 
                {
                 "covariance_type": "full",
                 "n_components": 1
                }
            },
         "return_only_object_with_seeds": True
        }     
}

# Path for graphCutInputConfig
graphCutInputConfig_JsonFilePath =  os.path.join(expFolder, "graphCutInputConfig.json")

In [63]:
def copyPatientToExpFolderAndGenerateAutoScribble(srcFolder, expFolder, patientName, expPatName,\
            patDataConfig, autoScribbleAndGCConfig, verbose):
    ctData,  ptData, gtData, softmaxData, predFromNN,\
        binLimit, bbVolume, numFGS, numBGS,\
        fgScribbleFromFGMissed, bgScribbleFromBGWrongC,\
        fgScribbleFromDefiniteFG, bgScribbleFromDefiniteBG,\
        fgScribble, bgScribble,\
        graphCutInputConfig, graphCutInputConfig_JsonFilePath\
       =  createGCInputUsingGT(patientName=patientName, srcFolder=srcFolder,\
            expFolder=expFolder, expPatName = expPatName,\
            patDataConfig=patDataConfig,\
            autoScribbleAndGCConfig = autoScribbleAndGCConfig,\
            verbose=verbose)
    return graphCutInputConfig, graphCutInputConfig_JsonFilePath

In [64]:
def generateSegmentFromScribble(graphCutInputConfig_JsonFilePath):
    from src.gcHelper import remote_generateGrahcutSegmentationAndDiceFromJson
    gcAndDiceResult = remote_generateGrahcutSegmentationAndDiceFromJson(graphCutInputConfig_JsonFilePath)
    print(gcAndDiceResult)
    return gcAndDiceResult

In [65]:
#This one creates separate segment from  label file and attach to common segmentation node
def createSegmentFromLabelFileInSlicer(folderPath, labelFileName, labelMapName,\
        parentSegmentationNode, colorTriple, sliceFillFlag, overallVisibility):
    #labelMapNode = slicer.util.addVolumeFromArray(labelData,ijkToRAS=None,
    #                      name=labelMapName,nodeClassName='vtkMRMLScalarVolumeNode')
    labelMapNode = slicer.util.loadLabelVolume(os.path.join(folderPath, labelFileName),\
            properties = {'name': labelMapName})
    slicer.modules.segmentations.logic().ImportLabelmapToSegmentationNode(labelMapNode, parentSegmentationNode)
    slicer.mrmlScene.RemoveNode(labelMapNode)
    parentSegmentationNode.GetSegmentation().GetSegment(labelMapName).SetColor(colorTriple[0], colorTriple[1],colorTriple[2])
    parentSegmentationNode.GetDisplayNode().SetSegmentVisibility2DFill(labelMapName,sliceFillFlag)
    parentSegmentationNode.GetDisplayNode().SetSegmentVisibility(labelMapName,overallVisibility)
    return labelMapName

In [66]:
def visualizeDataIn3DSlicer(graphCutInputConfig_JsonFilePath):
    with open(graphCutInputConfig_JsonFilePath) as fp:
            graphCutInputConfig = json.load(fp)
            fp.close()
    expFolder = graphCutInputConfig["expFolder"]
    expPatName = graphCutInputConfig["expPatName"]
    ctFileName = expPatName + patDataConfig['ctSuffix']
    ptFileName = expPatName + patDataConfig['ptSuffix']
    gtFileName = expPatName + patDataConfig['gtSuffix']
    softmaxFileName = expPatName + patDataConfig['softmaxSuffix']
    predFromNNFileName = expPatName + patDataConfig['predSuffix']
    fgScribbleFileName = expPatName + patDataConfig['fgScribbleSuffix']
    bgScribbleFileName = expPatName + patDataConfig['bgScribbleSuffix']
    gc_ct_FileName = expPatName + patDataConfig['ct_gCutSuffix']
    gc_pet_FileName = expPatName + patDataConfig['pet_gCutSuffix']
    gc_softmax_FileName = expPatName + patDataConfig['softmax_gCutSuffix']  
    
    import JupyterNotebooksLib as slicernb
    slicernb.ViewDisplay('OneUpRedSlice') #'FourUp', 'OneUp3D'
    slicer.mrmlScene.Clear()
    slicerNodes = {}
    
    ctVolumeNode = slicer.util.loadVolume(os.path.join(expFolder,ctFileName))
    ctVolumeNode.GetDisplayNode().SetAndObserveColorNodeID('vtkMRMLColorTableNodeGrey')
    slicerNodes['ctVolumeNode']= ctVolumeNode
    
    ptVolumeNode = slicer.util.loadVolume(os.path.join(expFolder,ptFileName))
    ptVolumeNode.GetDisplayNode().SetAndObserveColorNodeID('vtkMRMLPETProceduralColorNodePET-Heat')
    slicerNodes['ptVolumeNode']= ptVolumeNode
    
    softmaxVolumeNode = slicer.util.loadVolume(os.path.join(expFolder,softmaxFileName))
    softmaxVolumeNode.GetDisplayNode().SetAndObserveColorNodeID('vtkMRMLColorTableNodeBlue')
    slicerNodes['softmaxVolumeNode']= softmaxVolumeNode
    
    original_segmentationNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode", "original")
    gt_Name = createSegmentFromLabelFileInSlicer(expFolder, gtFileName, "gtLabelMap",\
              original_segmentationNode, (0,0,1), False, True)
    predNN_Name = createSegmentFromLabelFileInSlicer(expFolder, predFromNNFileName, "predNNLabelMap",\
              original_segmentationNode, (1,0,0), False, False)
    slicerNodes['original_segmentationNode']= original_segmentationNode
    slicerNodes['gt_Name']= gt_Name
    slicerNodes['predNN_Name']= predNN_Name
    
    fgScribble_segmentationNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode", "fgScribble")
    fgScribble_Name = createSegmentFromLabelFileInSlicer(expFolder, fgScribbleFileName,  "fgScribbleLabelMap",\
              fgScribble_segmentationNode, (0.15, 0.7, 0.8), True, True)
    slicerNodes['fgScribble_segmentationNode']= fgScribble_segmentationNode
    slicerNodes['fgScribble_Name']= fgScribble_Name
    
    bgScribble_segmentationNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode", "bgScribble")
    bgScribble_Name = createSegmentFromLabelFileInSlicer(expFolder, bgScribbleFileName, "bgScribbleLabelMap",\
              bgScribble_segmentationNode, (0.8, 0.7, 0.15), True, True)
    slicerNodes['bgScribble_segmentationNode']= bgScribble_segmentationNode
    slicerNodes['bgScribble_Name']= bgScribble_Name
    
    gcut_segmentationNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode", "graphcut")
    gc_softmax_Name = createSegmentFromLabelFileInSlicer(expFolder, gc_softmax_FileName, "gc_softmaxLabelMap",\
       gcut_segmentationNode, (0.3, 0.5, 0.7), False, True)    
    gc_ct_Name = createSegmentFromLabelFileInSlicer(expFolder, gc_ct_FileName, "gc_ctLabelMap",\
       gcut_segmentationNode, (0.7, 0.3, 0.5), False, False)    
    gc_pet_Name = createSegmentFromLabelFileInSlicer(expFolder, gc_pet_FileName, "gc_petLabelMap",\
       gcut_segmentationNode, (0.5, 0.7, 0.3), False, False)
    slicerNodes['gcut_segmentationNode']= gcut_segmentationNode
    slicerNodes['gc_softmax_Name']= gc_softmax_Name
    slicerNodes['gc_ct_Name']= gc_ct_Name
    slicerNodes['gc_pet_Name']= gc_pet_Name
    
    # slicer.util.setSliceViewerLayers(background=ctVolumeNode, foreground=ptVolumeNode, label=gt_segmentationNode\
    #                                 foregroundOpacity=0.5,labelOpacity=0.5)
    slicer.util.setSliceViewerLayers(background=softmaxVolumeNode, foreground=ptVolumeNode, foregroundOpacity=0.5)

    #reinforce visibility false:
    original_segmentationNode.GetDisplayNode().SetSegmentVisibility(predNN_Name,False)
    gcut_segmentationNode.GetDisplayNode().SetSegmentVisibility(gc_ct_Name,False)
    gcut_segmentationNode.GetDisplayNode().SetSegmentVisibility(gc_pet_Name,False)
    
    return slicerNodes
    

In [67]:
def updateScribbleFileFromSlicer(scribble_segmentationNode, scribble_Name, masterVolumeNode,\
                       folderPath, scribbleFileName):
    #Make sure the segment is visible; Since this is only segment no need to make others invisible
    scribble_segmentationNode.GetDisplayNode().SetSegmentVisibility(scribble_Name,True)
    #set references to masterVolumeNode
    scribble_segmentationNode.SetReferenceImageGeometryParameterFromVolumeNode(masterVolumeNode)
    #make temp labelvolumenode
    temp_labelmapVolumeNode = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLLabelMapVolumeNode')
    #export Segmentation to labelmapNode
    slicer.modules.segmentations.logic().ExportVisibleSegmentsToLabelmapNode(scribble_segmentationNode,\
    temp_labelmapVolumeNode, masterVolumeNode)
    #save labelmap to disk; before that copy original scribble to "old_" scribble file
    shutil.copy(os.path.join(folderPath, scribbleFileName), \
                os.path.join(folderPath, "old_"+ scribbleFileName))
    slicer.util.saveNode(temp_labelmapVolumeNode,os.path.join(folderPath, scribbleFileName))
    #Finally delete the temp labelvolumenode 
    slicer.mrmlScene.RemoveNode(temp_labelmapVolumeNode)

In [68]:
#srcFolder 
#expFolder 
#patientName = 'CHUM038'
#expPatName = "expPat"
#patDataConfig 
#autoScribbleAndGCConfig 
#graphCutInputConfig_JsonFilePath

class Dialog(qt.QDialog):
    """Dialog."""
    def __init__(self, srcFolder, expFolder, patientName, expPatName,\
                 patDataConfig, autoScribbleAndGCConfig, graphCutInputConfig_JsonFilePath,\
                 parent=None):
        """Initializer."""
        super().__init__(parent)
        self.graphCutInputConfig_JsonFilePath = graphCutInputConfig_JsonFilePath

        #Does graphCutInputConfig_JsonFile exists?
        # if os.path.exists(self.graphCutInputConfig_JsonFilePath):        
        #     if os.path.isfile(self.graphCutInputConfig_JsonFilePath): 
        #         with open(self.graphCutInputConfig_JsonFilePath) as fp:
        #             self.graphCutInputConfig = json.load(fp)
        #             fp.close()
        self.graphCutInputConfig = None        
        if self.graphCutInputConfig is None:
                self.srcFolder = srcFolder
                self.expFolder = expFolder
                self.patientName = patientName
                self.expPatName = expPatName
                self.patDataConfig = patDataConfig
                self.autoScribbleAndGCConfig = autoScribbleAndGCConfig         
        # else:
        #     self.srcFolder =  self.graphCutInputConfig["srcFolder"]
        #     self.expFolder = self.graphCutInputConfig["expFolder"]
        #     self.patientName =  self.graphCutInputConfig["patientName"]
        #     self.expPatName = self.graphCutInputConfig["expPatName"]
        #     self.patDataConfig = self.graphCutInputConfig["patDataConfig"]
        #     self.autoScribbleAndGCConfig = self.graphCutInputConfig["autoScribbleAndGCConfig"]
        
        #Clenup expFolder
        for f in glob.glob(expFolder + '/*', recursive=False):
            try:
                os.remove(f)
            except OSError as e:
                print("Error: %s : %s" % (f, e.strerror))
                
        self.scribbleInputChanged = False
        
        self.setWindowTitle('Fine tune GTVSegmentation dialog')
        #Components:        
        dlgLayout = qt.QVBoxLayout()
        
        #Scribble param
        dlgLayout.addWidget(qt.QLabel('<h4>Current auto-scribble params</h4>'))
        self.ScribbleFormLayout = qt.QFormLayout()
        ##BBox padding
        self.BBoxPadEntry = 'BBoxPadding:'
        self.BBoxPadEntryWidget = qt.QLineEdit(str(self.autoScribbleAndGCConfig['bbPad'])) 
        self.ScribbleFormLayout.addRow(self.BBoxPadEntry, self.BBoxPadEntryWidget)
        ##Dilation Diameter
        self.DilationDiaEntry = 'Dilation Dia:'
        self.DilationDiaEntryWidget = qt.QLineEdit(str(self.autoScribbleAndGCConfig['dilation_diam']))
        self.ScribbleFormLayout.addRow(self.DilationDiaEntry, self.DilationDiaEntryWidget)
        ##NumRegionAutoScribble
        self.NumRegionAutoScribbleEntry = 'Scribbles per region:'
        self.NumRegionAutoScribbleEntryWidget = \
            qt.QLineEdit(str(self.autoScribbleAndGCConfig['useAtmostNScribblesPerRegion']))
        self.ScribbleFormLayout.addRow(self.NumRegionAutoScribbleEntry, self.NumRegionAutoScribbleEntryWidget)
        dlgLayout.addLayout(self.ScribbleFormLayout)
        #auto scribble parameter update button
        self.UpdateScribbleParamButton = qt.QPushButton('UpdateScribbleParam')
        dlgLayout.addWidget(self.UpdateScribbleParamButton)
        
        #Labels
        dlgLayout.addWidget(qt.QLabel('<h4>' + 'srcDir: ' + os.path.basename(srcFolder)+ '</h4>'))
        dlgLayout.addWidget(qt.QLabel('<h4>' + 'expDir: ' + os.path.basename(expFolder)+ '</h4>'))
        #Instruction_1
        self.Instruction_1 = qt.QLabel('<h4>Choose a patient from srcDir</h4>')
        dlgLayout.addWidget(self.Instruction_1)  
        #Patient form
        self.PatientFormLayout = qt.QFormLayout()
        self.PatientEntry = 'Patient (e.g. CHGJ038):'
        self.PatientEntryWidget = qt.QLineEdit('')
        self.PatientFormLayout.addRow(self.PatientEntry, self.PatientEntryWidget)
        dlgLayout.addLayout(self.PatientFormLayout)
        #Load button 
        self.LoadButton = qt.QPushButton('Load patient, scribbles and segmentation')
        dlgLayout.addWidget(self.LoadButton )
        
        #Instruction_2 #'<h4>View segmentaion and add / modify scribble</h4>'
        self.Instruction_2 = qt.QLabel('')
        dlgLayout.addWidget(self.Instruction_2)
        #Save scribble and update segmentation button
        self.UpdateButton = qt.QPushButton('Save scribble and update Segmentation')
        dlgLayout.addWidget(self.UpdateButton)
        
        #Instruction_3 #'<h4>When satisfied, unload patient</h4>'
        self.Instruction_3 = qt.QLabel('')
        dlgLayout.addWidget(self.Instruction_3)
        #Unload patient button        
        self.UnloadButton = qt.QPushButton('Unload Patient')
        dlgLayout.addWidget(self.UnloadButton)
        
        #
        self.dlgLayout = dlgLayout
        self.setLayout(self.dlgLayout)
        
        #Set up handlers : Connection syntax:
        #widget.signal.connect(slot_function)
        self.UpdateScribbleParamButton.clicked.connect(self.UpdateScribbleParamButtonHandler)
        self.LoadButton.clicked.connect(self.loadButtonHandler)
        self.UpdateButton.clicked.connect(self.updateButtonHandler)
        self.UnloadButton.clicked.connect(self.unloadButtonHandler)
        
        #Initial button states:
        self.UpdateScribbleParamButton.setEnabled(True)
        self.LoadButton.setEnabled(True)
        self.UpdateButton.setEnabled(False)
        self.UnloadButton.setEnabled(False)
        
    def UpdateScribbleParamButtonHandler(self):
        #https://stackoverflow.com/questions/57171474/get-text-from-a-qlineedit-in-a-qformlayout-using-itemat
        #Hard learn after speding 6 hours: For slicer QLineEdit(), text() is not valid
        # as it is not a function but just a public member.
        #Read updated scribble parameter
        self.autoScribbleAndGCConfig['bbPad'] = int(self.BBoxPadEntryWidget.text)
        self.autoScribbleAndGCConfig['dilation_diam'] = int(self.DilationDiaEntryWidget.text)
        self.autoScribbleAndGCConfig['useAtmostNScribblesPerRegion'] \
            = int(self.NumRegionAutoScribbleEntryWidget.text)
        self.scribbleInputChanged = True 
        
    def loadButtonHandler(self):                
        #Read patient name from patient entry form        
        self.patientName = self.PatientEntryWidget.text
        #Generate new scribble and graph-cut under following condition
#         print(self.scribbleInputChanged)
#         print('Entered ', self.PatientEntryWidget.text)
#         print('Old ', self.patientName)
#         self.PatientName = self.PatientEntryWidget.text
#         print('New ', self.patientName)
#         print(self.graphCutInputConfig["patientName"])
        if True == self.scribbleInputChanged\
            or self.graphCutInputConfig is None\
            or self.patientName !=  self.graphCutInputConfig["patientName"]:
            print("Triggering new scribble creation")
            ctData,  ptData, gtData, softmaxData, predFromNN,\
            binLimit, bbVolume, numFGS, numBGS,\
            fgScribbleFromFGMissed, bgScribbleFromBGWrongC,\
            fgScribbleFromDefiniteFG, bgScribbleFromDefiniteBG,\
            fgScribble, bgScribble,\
            self.graphCutInputConfig, self.graphCutInputConfig_JsonFilePath\
            =  createGCInputUsingGT(self.patientName, self.srcFolder,\
                self.expFolder, self.expPatName,\
                self.patDataConfig,self.autoScribbleAndGCConfig,\
                verbose=False)
            #######################################
            print("Triggering new graphcut creation")
            # from src.gcHelper import remote_generateGrahcutSegmentationAndDiceFromJson
            gcAndDiceResult = \
                remote_generateGrahcutSegmentationAndDiceFromJson(self.graphCutInputConfig_JsonFilePath)
            print(gcAndDiceResult)
            ####################################
            #Mark scribbleInputChanged as false
            self.scribbleInputChanged = False
        else:
            print("NOT Triggering new graphcut or scribble  creation")
        ###########################################################################
        #Load result (newly created or cached) in slicer
        self.slicerNodes = visualizeDataIn3DSlicer(self.graphCutInputConfig_JsonFilePath)
        ############################################################################
        #Update dialog UI
        self.Instruction_2.setText('<h4>' + self.patientName +\
            ' loaded. View and edit scribbles.</h4>')
        self.Instruction_3.setText('<h4>Unload ' + self.patientName + '</h4>')
        #Update button states
        self.UpdateScribbleParamButton.setEnabled(False)
        self.LoadButton.setEnabled(False)
        self.UpdateButton.setEnabled(True)
        self.UnloadButton.setEnabled(True)
        
    def updateButtonHandler(self):
        fgScribbleFileName = self.expPatName + self.patDataConfig['fgScribbleSuffix']
        bgScribbleFileName = self.expPatName + self.patDataConfig['bgScribbleSuffix']
        # ########################## Save update FG_scribble ########################
        updateScribbleFileFromSlicer(self.slicerNodes['fgScribble_segmentationNode'],\
            self.slicerNodes['fgScribble_Name'], self.slicerNodes['softmaxVolumeNode'],\
            self.expFolder, fgScribbleFileName) 
        # ########################## Save update BG_scribble ########################
        updateScribbleFileFromSlicer(self.slicerNodes['bgScribble_segmentationNode'],\
            self.slicerNodes['bgScribble_Name'], self.slicerNodes['softmaxVolumeNode'],\
            self.expFolder, bgScribbleFileName) 
        #######################################
        print("Triggering new graphcut creation")
        # from src.gcHelper import remote_generateGrahcutSegmentationAndDiceFromJson
        gcAndDiceResult = \
         remote_generateGrahcutSegmentationAndDiceFromJson(self.graphCutInputConfig_JsonFilePath)
        print(gcAndDiceResult)
        ####################################
        #Load result (newly created or cached) in slicer
        self.slicerNodes = visualizeDataIn3DSlicer(self.graphCutInputConfig_JsonFilePath)
        print('segmentation updated')
        
    def unloadButtonHandler(self):        
        slicer.mrmlScene.Clear()
        self.slicerNodes = {}   
        self.Instruction_2.setText('')
        self.Instruction_3.setText('')
        #Update button states
        self.UpdateScribbleParamButton.setEnabled(True)
        self.LoadButton.setEnabled(True)
        self.UpdateButton.setEnabled(False)
        self.UnloadButton.setEnabled(False)

In [69]:
dlg = Dialog(srcFolder, expFolder, patientName, expPatName,\
     patDataConfig, autoScribbleAndGCConfig, graphCutInputConfig_JsonFilePath)
dlg.show()

Triggering new scribble creation
Triggering new graphcut creation
Starting action :  imcut
starting connection to ('127.0.0.1', 65432)
sending b'\x00h{"byteorder": "little", "content-type": "text/json", "content-encoding": "utf-8", "content-length": 127}{"action": "imcut", "value": "/home/user/DMML/Data/PlayDataManualSegmentation/AutoScribbleExperiment/graphCutInputConfig.json"}' to ('127.0.0.1', 65432)
received text/json response from ('127.0.0.1', 65432)
graphCut success : True
closing connection to ('127.0.0.1', 65432)
{'successFlag': True, 'patientName': 'CHGJ038', 'gcPath_ct': '/home/user/DMML/Data/PlayDataManualSegmentation/AutoScribbleExperiment/expPat_ct_gCut.nii.gz', 'gcPath_pet': '/home/user/DMML/Data/PlayDataManualSegmentation/AutoScribbleExperiment/expPat_pet_gCut.nii.gz', 'gcPath_softmax': '/home/user/DMML/Data/PlayDataManualSegmentation/AutoScribbleExperiment/expPat_softmax_gCut.nii.gz', 'originalDice': 0.8771493378210288, 'numFGS': 10, 'numBGS': 10, 'gcDice_ct': 0.444520

####  Prepare initial random scribble and run graphcut on CT / PET / Softmax (remotely as we are in slicer 4.11 environment  where imcut() could not be implemented)

In [ ]:
# print('Running single experiment on individual patient')

# ctData,  ptData, gtData, softmaxData, predFromNN,\
#     binLimit, bbVolume, numFGS, numBGS,\
#     fgScribbleFromFGMissed, bgScribbleFromBGWrongC,\
#     fgScribbleFromDefiniteFG, bgScribbleFromDefiniteBG,\
#     fgScribble, bgScribble,\
#     graphCutInputConfig, graphCutInputConfig_JsonFilePath\
#    =  createGCInputUsingGT(patientName=patientName, srcFolder=srcFolder,\
#         expFolder=expFolder, expPatName = expPatName,\
#         patDataConfig=patDataConfig,\
#         autoScribbleAndGCConfig = autoScribbleAndGCConfig,\
#         verbose=verbose)

# from src.gcHelper import remote_generateGrahcutSegmentationAndDiceFromJson    
# gcAndDiceResult = remote_generateGrahcutSegmentationAndDiceFromJson(graphCutInputConfig_JsonFilePath)
# print(gcAndDiceResult)

In [ ]:
# graphCutInputConfig_JsonFilePath = "/home/user/DMML/Data/PlayDataManualSegmentation/AutoScribbleExperiment/graphCutInputConfig.json"
# from src.gcHelper import remote_generateGrahcutSegmentationAndDiceFromJson    
# gcAndDiceResult = remote_generateGrahcutSegmentationAndDiceFromJson(graphCutInputConfig_JsonFilePath)
# print(gcAndDiceResult)

In [ ]:
# ctFileName = expPatName + patDataConfig['ctSuffix']
# ptFileName = expPatName + patDataConfig['ptSuffix']
# gtFileName = expPatName + patDataConfig['gtSuffix']
# softmaxFileName = expPatName + patDataConfig['softmaxSuffix']
# predFromNNFileName = expPatName + patDataConfig['predSuffix']
# fgScribbleFileName = expPatName + patDataConfig['fgScribbleSuffix']
# bgScribbleFileName = expPatName + patDataConfig['bgScribbleSuffix']
# gc_ct_FileName = expPatName + patDataConfig['ct_gCutSuffix']
# gc_pet_FileName = expPatName + patDataConfig['pet_gCutSuffix']
# gc_softmax_FileName = expPatName + patDataConfig['softmax_gCutSuffix']
# print(expFolder)
# print(ctFileName)
# print(ptFileName)
# print(gtFileName)
# print(softmaxFileName)
# print(predFromNNFileName)
# print(fgScribbleFileName)
# print(bgScribbleFileName)
# print(gc_ct_FileName)
# print(gc_pet_FileName)
# print(gc_softmax_FileName)

In [ ]:
# import JupyterNotebooksLib as slicernb
# slicernb.ViewDisplay('OneUpRedSlice') #'FourUp', 'OneUp3D'
# slicer.mrmlScene.Clear()

In [ ]:
# ctVolumeNode = slicer.util.loadVolume(os.path.join(expFolder,ctFileName))
# ctVolumeNode.GetDisplayNode().SetAndObserveColorNodeID('vtkMRMLColorTableNodeGrey')

# ptVolumeNode = slicer.util.loadVolume(os.path.join(expFolder,ptFileName))
# ptVolumeNode.GetDisplayNode().SetAndObserveColorNodeID('vtkMRMLPETProceduralColorNodePET-Heat')

# softmaxVolumeNode = slicer.util.loadVolume(os.path.join(expFolder,softmaxFileName))
# softmaxVolumeNode.GetDisplayNode().SetAndObserveColorNodeID('vtkMRMLColorTableNodeBlue')

# original_segmentationNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode", "original")
# gt_Name = createSegmentFromLabelFile(expFolder, gtFileName, "gtLabelMap",\
#           original_segmentationNode, (0,0,1), False, True)
# predNN_Name = createSegmentFromLabelFile(expFolder, predFromNNFileName, "predNNLabelMap",\
#           original_segmentationNode, (1,0,0), False, False)


# fgScribble_segmentationNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode", "fgScribble")
# fgScribble_Name = createSegmentFromLabelFile(expFolder, fgScribbleFileName,  "fgScribbleLabelMap",\
#           fgScribble_segmentationNode, (0.15, 0.7, 0.8), True, True)

# bgScribble_segmentationNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode", "bgScribble")
# bgScribble_Name = createSegmentFromLabelFile(expFolder, bgScribbleFileName, "bgScribbleLabelMap",\
#           bgScribble_segmentationNode, (0.8, 0.7, 0.15), True, True)

# gcut_segmentationNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode", "graphcut")
# gc_softmax_Name = createSegmentFromLabelFile(expFolder, gc_softmax_FileName, "gc_softmaxLabelMap",\
#    gcut_segmentationNode, (0.3, 0.5, 0.7), False, True)    
# gc_ct_Name = createSegmentFromLabelFile(expFolder, gc_ct_FileName, "gc_ctLabelMap",\
#    gcut_segmentationNode, (0.7, 0.3, 0.5), False, False)    
# gc_pet_Name = createSegmentFromLabelFile(expFolder, gc_pet_FileName, "gc_petLabelMap",\
#    gcut_segmentationNode, (0.5, 0.7, 0.3), False, False)

# # slicer.util.setSliceViewerLayers(background=ctVolumeNode, foreground=ptVolumeNode, label=gt_segmentationNode\
# #                                 foregroundOpacity=0.5,labelOpacity=0.5)
# slicer.util.setSliceViewerLayers(background=softmaxVolumeNode, foreground=ptVolumeNode, foregroundOpacity=0.5)

# #reinforce visibility false:
# original_segmentationNode.GetDisplayNode().SetSegmentVisibility(predNN_Name,False)
# gcut_segmentationNode.GetDisplayNode().SetSegmentVisibility(gc_ct_Name,False)
# gcut_segmentationNode.GetDisplayNode().SetSegmentVisibility(gc_pet_Name,False)


In [ ]:
# # def updateScribbleFile(scribble_segmentationNode, scribble_Name, masterVolumeNode,\
# #                        folderPath, scribbleFileName):
# #     #Make sure the segment is visible; Since this is only segment no need to make others invisible
# #     scribble_segmentationNode.GetDisplayNode().SetSegmentVisibility(scribble_Name,True)
# #     #set references to masterVolumeNode
# #     scribble_segmentationNode.SetReferenceImageGeometryParameterFromVolumeNode(masterVolumeNode)
# #     #make temp labelvolumenode
# #     temp_labelmapVolumeNode = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLLabelMapVolumeNode')
# #     #export Segmentation to labelmapNode
# #     slicer.modules.segmentations.logic().ExportVisibleSegmentsToLabelmapNode(scribble_segmentationNode,\
# #     temp_labelmapVolumeNode, masterVolumeNode)
# #     #save labelmap to disk; before that copy original scribble to "old_" scribble file
# #     shutil.copy(os.path.join(folderPath, scribbleFileName), \
# #                 os.path.join(folderPath, "old_"+ scribbleFileName))
# #     slicer.util.saveNode(temp_labelmapVolumeNode,os.path.join(folderPath, scribbleFileName))
# #     #Finally delete the temp labelvolumenode 
# #     slicer.mrmlScene.RemoveNode(temp_labelmapVolumeNode)

# ########################## Save update FG_scribble ########################
# updateScribbleFile(fgScribble_segmentationNode, fgScribble_Name, softmaxVolumeNode,\
#                        expFolder, fgScribbleFileName) 

# ########################## Save update BG_scribble ########################
# updateScribbleFile(bgScribble_segmentationNode, bgScribble_Name, softmaxVolumeNode,\
#                        expFolder, bgScribbleFileName) 



In [ ]:
# #Re run graphcut updated scribble 
# from src.gcHelper import remote_generateGrahcutSegmentationAndDiceFromJson    
# gcAndDiceResult = remote_generateGrahcutSegmentationAndDiceFromJson(graphCutInputConfig_JsonFilePath)
# print(gcAndDiceResult)

In [ ]:
# slicer.mrmlScene.Clear()

In [ ]:
# #Save scribble
# Export a selection of segments (identified by their names):
# segmentNames = ["Prostate", "Urethra"]
# segmentIds = vtk.vtkStringArray()
# for segmentName in segmentNames:
#     segmentId = segmentationNode.GetSegmentation().GetSegmentIdBySegmentName(segmentName)
#     segmentIds.InsertNextValue(segmentId)
# slicer.vtkSlicerSegmentationsModuleLogic.ExportSegmentsToLabelmapNode(segmentationNode,\
#             segmentIds, labelmapVolumeNode, referenceVolumeNode)

########################## Save update FG_scribble ########################
# fgScribble_segmentationNode.GetDisplayNode().SetSegmentVisibility(fgScribble_Name,True)
# #set references to masterVolumeNode
# fgScribble_segmentationNode.SetReferenceImageGeometryParameterFromVolumeNode(softmaxVolumeNode)
# #make temp labelvolumenode
# temp_labelmapVolumeNode = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLLabelMapVolumeNode')
# #export Segmentation to labelmapNode
# slicer.modules.segmentations.logic().ExportVisibleSegmentsToLabelmapNode(fgScribble_segmentationNode,\
#     temp_labelmapVolumeNode, softmaxVolumeNode)
# #save labelmap to disk; before that copy original scribble to "old_" scribble file
# shutil.copy(os.path.join(expFolder, fgScribbleFileName), \
#                 os.path.join(expFolder, "old_"+fgScribbleFileName))
# slicer.util.saveNode(temp_labelmapVolumeNode,os.path.join(expFolder, fgScribbleFileName))
# #Finally delete the temp labelvolumenode 
# slicer.mrmlScene.RemoveNode(temp_labelmapVolumeNode)

########################## Save update BG_scribble ########################
# bgScribble_segmentationNode.GetDisplayNode().SetSegmentVisibility(bgScribble_Name,True)
# #set references to masterVolumeNode
# bgScribble_segmentationNode.SetReferenceImageGeometryParameterFromVolumeNode(softmaxVolumeNode)
# #make temp labelvolumenode
# temp_labelmapVolumeNode = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLLabelMapVolumeNode')
# #export Segmentation to labelmapNode
# slicer.modules.segmentations.logic().ExportVisibleSegmentsToLabelmapNode(bgScribble_segmentationNode,\
#     temp_labelmapVolumeNode, softmaxVolumeNode)
# #save labelmap to disk
# shutil.copy(os.path.join(expFolder, bgScribbleFileName), \
#                 os.path.join(expFolder, "old_"+bgScribbleFileName))
# slicer.util.saveNode(temp_labelmapVolumeNode,os.path.join(expFolder, bgScribbleFileName))
# #Finally delete the temp labelvolumenode 
# slicer.mrmlScene.RemoveNode(temp_labelmapVolumeNode)
###########################################################################

In [ ]:
# #This one creates separate segmentation nodes from each label file : Not used any more
# def createSegmentationNodeFromLabelFile(folderPath, labelFileName, labelMapName,\
#         segmentationNodeName, colorTriple, sliceFillFlag, overallVisibility=True):
#     #labelMapNode = slicer.util.addVolumeFromArray(labelData,ijkToRAS=None,
#     #                      name=labelMapName,nodeClassName='vtkMRMLScalarVolumeNode')
#     labelMapNode = slicer.util.loadLabelVolume(os.path.join(folderPath, labelFileName),\
#             properties = {'name': labelMapName})
#     segmentationNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode", segmentationNodeName)
#     slicer.modules.segmentations.logic().ImportLabelmapToSegmentationNode(labelMapNode, segmentationNode)
#     slicer.mrmlScene.RemoveNode(labelMapNode)
#     segmentationNode.GetSegmentation().GetSegment(labelMapName).SetColor(colorTriple[0], colorTriple[1],colorTriple[2])
#     if True == sliceFillFlag:
#         segmentationNode.GetDisplayNode().SetVisibility2DFill(1)
#         #segmentationNode.SetSegmentVisibility2DFill('Segment_1',True)
#     else:
#         segmentationNode.GetDisplayNode().SetVisibility2DFill(0)
#         #segmentationNode.SetSegmentVisibility2DFill('Segment_1',False)
#     #set overall visibility
#     segmentationNode.GetDisplayNode().SetVisibility(overallVisibility)
#     return segmentationNode

In [ ]:
# ctVolumeNode = slicer.util.loadVolume(os.path.join(expFolder,ctFileName))
# ctVolumeNode.GetDisplayNode().SetAndObserveColorNodeID('vtkMRMLColorTableNodeGrey')

# ptVolumeNode = slicer.util.loadVolume(os.path.join(expFolder,ptFileName))
# ptVolumeNode.GetDisplayNode().SetAndObserveColorNodeID('vtkMRMLPETProceduralColorNodePET-Heat')

# softmaxVolumeNode = slicer.util.loadVolume(os.path.join(expFolder,softmaxFileName))
# softmaxVolumeNode.GetDisplayNode().SetAndObserveColorNodeID('vtkMRMLColorTableNodeBlue')

# gt_segmentationNode = createSegmentationNodeFromLabelFile(expFolder, gtFileName, \
#                          "gtLabelMap", "gtSeg", (0,0,1), False, True)
# predNN_segmentationNode = createSegmentationNodeFromLabelFile(expFolder, predFromNNFileName, \
#                          "predNNLabelMap", "predNNSeg", (1,0,0), False, False)

# fgScribble_segmentationNode = createSegmentationNodeFromLabelFile(expFolder, fgScribbleFileName, \
#                          "fgScribbleLabelMap", "fgScribbleSeg", (0.15, 0.7, 0.8), True, True)
# bgScribble_segmentationNode = createSegmentationNodeFromLabelFile(expFolder, bgScribbleFileName, \
#                          "bgScribbleLabelMap", "bgScribbleSeg", (0.8, 0.7, 0.15), True, True)

# gc_softmax_segmentationNode = createSegmentationNodeFromLabelFile(expFolder,\
#   gc_softmax_FileName, "gc_softmaxLabelMap", "gc_softmaxSeg", \
#     (0.3, 0.5, 0.7), False, True)

# gc_ct_segmentationNode = createSegmentationNodeFromLabelFile(expFolder,\
#   gc_ct_FileName, "gc_ctLabelMap", "gc_ctSeg", \
#     (0.7, 0.3, 0.5), False, False)

# gc_pet_segmentationNode = createSegmentationNodeFromLabelFile(expFolder,\
#   gc_pet_FileName, "gc_petLabelMap", "gc_petSeg", \
#     (0.5, 0.7, 0.3), False, False)

# # slicer.util.setSliceViewerLayers(background=ctVolumeNode, foreground=ptVolumeNode, label=gt_segmentationNode\
# #                                 foregroundOpacity=0.5,labelOpacity=0.5)
# slicer.util.setSliceViewerLayers(background=softmaxVolumeNode, foreground=ptVolumeNode, foregroundOpacity=0.5)

In [ ]:
# gt_segmentationNode = createSegmentationNodeFromLabelFile(expFolder, gtFileName, \
#                          "gtLabelMap", "gtSeg", (0,0,1), False)
# predNN_segmentationNode = createSegmentationNodeFromLabelFile(expFolder, predFromNNFileName, \
#                          "predNNLabelMap", "predNNSeg", (1,0,0), False)
# fgScribble_segmentationNode = createSegmentationNodeFromLabelFile(expFolder, fgScribbleFileName, \
#                          "fgScribbleLabelMap", "fgScribbleSeg", (0.15, 0.7, 0.8), True)
# bgScribble_segmentationNode = createSegmentationNodeFromLabelFile(expFolder, bgScribbleFileName, \
#                          "bgScribbleLabelMap", "bgScribbleSeg", (0.8, 0.7, 0.15), True)

# # slicer.util.setSliceViewerLayers(background=ctVolumeNode, foreground=ptVolumeNode, label=gt_segmentationNode\
# #                                 foregroundOpacity=0.5,labelOpacity=0.5)


In [ ]:
#Python interactor output
# >>> expPat_pt_VolumeNode = getNode('expPat_pt')
# >>> type(expPat_pt_VolumeNode)
# <class 'MRMLCorePython.vtkMRMLScalarVolumeNode'>
# >>> expPat_pt_DisplayNode = expPat_pt_VolumeNode.GetDisplayNode()
# >>> type(expPat_pt_DisplayNode)
# <class 'MRMLCorePython.vtkMRMLScalarVolumeDisplayNode'>
# >>> expPat_pt_ColorNode = expPat_pt_DisplayNode.GetColorNode()
# >>> type(expPat_pt_ColorNode)
# <class 'MRMLCorePython.vtkMRMLColorTableNode'>
# >>> expPat_pt_CurrentLUT = expPat_pt_ColorNode.GetLookupTable()
# >>> type(expPat_pt_CurrentLUT)
# <class 'vtkCommonCorePython.vtkLookupTable'>

# >>> indexL = expPat_pt_CurrentLUT.GetIndexedLookup()
# >>> type(indexL)
# <class 'int'>
# >>> print(indexL)
# 0
# >>> expPat_pt_CurrentLUT.SetIndexedLookup(3)
# >>> expPat_pt_CurrentLUT = expPat_pt_ColorNode.GetLookupTable()
# >>> print(expPat_pt_CurrentLUT.GetIndexedLookup())
# 3
# >>> getNode('expPat_pt').GetDisplayNode().GetColorNode().IsTypeOf('vtkMRMLColorTableNode')
# 1
# >>> getNode('expPat_pt').GetDisplayNode().GetColorNode().IsTypeOf('vtkMRMLProceduralColorNode')
# 0

In [ ]:
# #Experimental read write json file
# with open("/home/user/DMML/Data/PlayDataManualSegmentation/AutoScribbleExperiment/expPat_graphCutInput.json") as fp:
#         gcSConfig = json.load(fp)
#         fp.close() 
# srcFolder = gcSConfig["srcFolder" ]
# patDataConfig = gcSConfig["patDataConfig" ]
# autoScribbleAndGCConfig = gcSConfig['autoScribbleAndGCConfig']
# expFolder = patDataConfig["expFolder"]

# print(srcFolder)
# print(expFolder)
# print(autoScribbleAndGCConfig)
# print(patDataConfig['labels_to_train'])

# #Make changes

# new_expFolder = "/home/user/DMML/Data/PlayDataManualSegmentation/ManualScribble"
# new_patDataConfig = patDataConfig
# new_patDataConfig["expFolder"]= new_expFolder

# new_srcFolder = "/home/user/DMML/Data/HeadNeck_PET_CT/nnUnet_3dfullres/training_gtvs_withSoftmax"
# new_gcSConfig = gcSConfig
# new_gcSConfig["srcFolder" ] = new_srcFolder
# new_gcSConfig["patDataConfig" ] = new_patDataConfig
# with open("/home/user/DMML/Data/PlayDataManualSegmentation/AutoScribbleExperiment/modified_expPat_graphCutInput.json", 'w') as fp:
#         json.dump(new_gcSConfig, fp, ) #, indent='' #, indent=4
#         fp.close()
        
# #Let us test
# #Experimental read write json file
# with open("/home/user/DMML/Data/PlayDataManualSegmentation/AutoScribbleExperiment/modified_expPat_graphCutInput.json") as fp:
#         gcSConfig2 = json.load(fp)
#         fp.close() 
# srcFolder2 = gcSConfig2["srcFolder" ]
# patDataConfig2 = gcSConfig2["patDataConfig" ]
# autoScribbleAndGCConfig2 = gcSConfig2['autoScribbleAndGCConfig']
# expFolder2 = patDataConfig2["expFolder"]

# print('####################')
# print(srcFolder2)
# print(expFolder2)
# print(autoScribbleAndGCConfig2)
# print(patDataConfig2['labels_to_train'])